# 5.5 读写文件

## 5.5.1 加载和保存张量
对于单个张量，我们可以直接调用load和save函数分别读写它们。

这两个函数都要求我们提供一个名称，save要求将要保存的变量作为输入。

In [3]:
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')

In [4]:
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

In [5]:
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [6]:
# 可以写入或读取从字符串映射到张量的字典
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2


{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

## 5.5.2 加载和保存模型参数

In [9]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)
net,X,Y.shape,Y

(MLP(
   (hidden): Linear(in_features=20, out_features=256, bias=True)
   (output): Linear(in_features=256, out_features=10, bias=True)
 ),
 tensor([[-0.1839, -2.4737, -1.4116,  0.1050,  0.3957,  0.0476, -0.0105,  1.9014,
          -1.8765, -0.7378,  0.0521, -0.7170, -0.1130,  0.6201,  0.5541, -0.9708,
           0.4555,  0.7925,  1.0275, -0.8550],
         [ 0.1904,  1.5227, -0.1923, -0.3002, -1.4130,  0.2606,  0.2048, -0.6006,
          -1.1079, -0.5616,  0.6885,  0.6127,  0.4538, -1.1915, -1.4091,  0.4521,
           0.5807,  0.3451,  0.6916, -0.5331]]),
 torch.Size([2, 10]),
 tensor([[-0.1504,  0.0997, -0.0231, -0.2205,  0.0336,  0.0394, -0.2437, -0.0926,
           0.1436, -0.1431],
         [ 0.1064, -0.3339, -0.0427, -0.0966, -0.0355,  0.1874, -0.1459, -0.1073,
           0.1408, -0.1012]], grad_fn=<AddmmBackward0>))

In [10]:
# 将模型的参数存储为一个叫做“mlp.params”的文件。
torch.save(net.state_dict(), 'mlp.params')


### eval()
在Python中evel()函数的语法格式为eval(expression, globals=None, locals=None)，注意后面还有globals参数和locals参数。eval()函数用于执行一个字符串表达式，并且返回该表达式的值。与eval相近的有exec函数，

In [20]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()
# net.state_dict() == clone.state_dict

RuntimeError: Boolean value of Tensor with more than one value is ambiguous

In [22]:
# 由于两个实例具有相同的模型参数，
# 在输入相同的X时，两个实例的计算结果应该相同。
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

## 5.5.3 小结

* `save`和`load`函数可用于张量对象的文件读写。
* 我们可以通过参数字典保存和加载网络的全部参数。
* 保存结构必须在代码中完成，而不是在参数中完成。
 
## 5.5.4 练习

1. 即使不需要将经过训练的模型部署到不同的设备上，存储模型参数还有什么实际的好处？
1. 假设我们只想复用网络的一部分，以将其合并到不同的网络结构中。比如说，如果你想在一个新的网络中使用之前网络的前两层，你该怎么做？
1. 如何同时保存网络结构和参数？你会对结构加上什么限制？